<a href="https://colab.research.google.com/github/otumina-collab/POO-Actividad_N-_5/blob/main/Actividad_N%C2%B05_POO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from ipywidgets import Text, Button, Output, VBox, HBox, Layout, Dropdown, HTML
from IPython.display import display, clear_output
import csv
import os
import json

# Definición de la clase CRUDArchivo (para persistencia)
class CRUDArchivo:
    def __init__(self, archivo):
        self.archivo = archivo
        if not os.path.exists(self.archivo):
            with open(self.archivo, "w") as f:
                pass

    def create(self, texto):
        with open(self.archivo, "a") as f:
            f.write(texto + "\n")

    def read(self):
        if not os.path.exists(self.archivo):
            return ""
        with open(self.archivo, "r") as f:
            return f.read()

    def update(self, linea, nuevo_texto):
        with open(self.archivo, "r") as f:
            lineas = f.readlines()

        if linea < 1 or linea > len(lineas):
            return False

        lineas[linea - 1] = nuevo_texto + "\n"

        with open(self.archivo, "w") as f:
            f.writelines(lineas)

        return True

    def delete(self, linea):
        with open(self.archivo, "r") as f:
            lineas = f.readlines()

        if linea < 1 or linea > len(lineas):
            return False

        del lineas[linea - 1]

        with open(self.archivo, "w") as f:
            f.writelines(lineas)

        return True

# --- Lista Global de Contactos (Inicializada)
contacts = []

# Instancia de CRUDArchivo para gestionar los contactos en un archivo
contact_file_manager = CRUDArchivo('contacts.txt')

#Funciones de Persistencia --- (Ahora usan CRUDArchivo)
def save_contacts_to_file():
    global contacts
    # Eliminar y recrear el archivo para sobrescribir con el estado actual
    if os.path.exists(contact_file_manager.archivo):
        os.remove(contact_file_manager.archivo)
    contact_file_manager.__init__(contact_file_manager.archivo)

    for contact in contacts:
        contact_file_manager.create(json.dumps(contact))

def load_contacts_from_file():
    global contacts
    contacts = []
    content = contact_file_manager.read()
    if content:
        for line in content.splitlines():
            if line.strip():
                try:
                    contacts.append(json.loads(line))
                except json.JSONDecodeError:
                    print(f"Warning: Could not decode JSON line: {line}")
    _update_contact_selector()
    _display_contacts(contacts)

# Campos de entrada
name_input = Text(description='Name:', placeholder='Full Name', layout=Layout(width='50%'))
phone_input = Text(description='Phone:', placeholder='123-456-7890', layout=Layout(width='50%'))
email_input = Text(description='Email:', placeholder='email@example.com', layout=Layout(width='50%'))
contact_id_input = Text(description='Contact ID:', placeholder='ID to Update/Delete', layout=Layout(width='50%'))

# Nuevo widget de entrada de búsqueda
search_input = Text(description='Search:', placeholder='Search by name or phone', layout=Layout(width='50%'))

# Desplegable para la selección de contactos
contact_selector_dropdown = Dropdown(
    options=[('Select a contact', None)],
    value=None,
    description='Select:',
    layout=Layout(width='50%')
)

# Botones para operaciones CRUD, búsqueda y persistencia
add_button = Button(description=' Create Contact', button_style='success', layout=Layout(width='auto'))
update_button = Button(description=' Update Contact', button_style='success', layout=Layout(width='auto'))
delete_button = Button(description=' Delete Contact', button_style='success', layout=Layout(width='auto'))
search_button = Button(description=' Search Contact', button_style='warning', layout=Layout(width='auto'))
clear_button = Button(description=' Clear Fields', button_style='warning', layout=Layout(width='auto'))
save_button = Button(description=' Save Contacts', button_style='primary', layout=Layout(width='auto'))
load_button = Button(description=' Load Contacts', button_style='primary', layout=Layout(width='auto'))
export_csv_button = Button(description=' Read Contact', button_style='success', layout=Layout(width='auto'))

# Área de salida para mensajes
output_area = Output()

#Funciones Auxiliares
def _update_contact_selector():
    # Actualiza las opciones del Dropdown con los contactos actuales
    if not contacts:
        contact_selector_dropdown.options = [('Select a contact', None)]
        contact_selector_dropdown.value = None
    else:
        options = [('Select a contact', None)]
        for i, contact in enumerate(contacts):
            options.append((f"{contact['name']} ({contact['phone']}) - ID: {i}", i))
        contact_selector_dropdown.options = options
        contact_selector_dropdown.value = None

def _display_contacts(contact_list_to_display):
    with output_area:
        clear_output()
        if not contact_list_to_display:
            print("No contacts to display.")
        else:
            print("--- Contact List ---")
            for i, contact in enumerate(contact_list_to_display):
                print(f"ID: {i}, Name: {contact['name']}, Phone: {contact['phone']}, Email: {contact['email']}")
            print("-------------------------")

def _clear_input_fields():
    name_input.value = ''
    phone_input.value = ''
    email_input.value = ''
    contact_id_input.value = ''
    search_input.value = ''

#Handlers de Eventos para Botones y Dropdown
def on_add_clicked(b):
    with output_area:
        clear_output()
        name = name_input.value.strip()
        phone = phone_input.value.strip()
        email = email_input.value.strip()

        if name and phone:
            contacts.append({'name': name, 'phone': phone, 'email': email})
            print(f"Contact '{name}' added successfully.")
            save_contacts_to_file()
            _clear_input_fields()
            _update_contact_selector()
            _display_contacts(contacts)
        else:
            print("Error: Name and Phone are required fields.")

def on_update_clicked(b):
    with output_area:
        clear_output()
        contact_id_str = contact_id_input.value.strip()
        name = name_input.value.strip()
        phone = phone_input.value.strip()
        email = email_input.value.strip()

        if contact_id_str.isdigit() and (name or phone or email):
            contact_id = int(contact_id_str)
            if 0 <= contact_id < len(contacts):
                if name: contacts[contact_id]['name'] = name
                if phone: contacts[contact_id]['phone'] = phone
                if email: contacts[contact_id]['email'] = email
                print(f"Contact ID {contact_id} updated successfully.")
                save_contacts_to_file()
                _clear_input_fields()
                _update_contact_selector()
                _display_contacts(contacts)
            else:
                print(f"Error: Contact ID '{contact_id_str}' out of range.")
        else:
            print("Error: Enter a valid contact ID and at least one field to update.")

def on_delete_clicked(b):
    with output_area:
        clear_output()
        contact_id_str = contact_id_input.value.strip()

        if contact_id_str.isdigit():
            contact_id = int(contact_id_str)
            if 0 <= contact_id < len(contacts):
                deleted_contact = contacts.pop(contact_id)
                print(f"Contact '{deleted_contact['name']}' (ID: {contact_id}) deleted successfully.")
                save_contacts_to_file()
                _clear_input_fields()
                _update_contact_selector()
                _display_contacts(contacts)
            else:
                print(f"Error: Contact ID '{contact_id_str}' out of range.")
        else:
            print("Error: Enter a valid contact ID to delete.")

def on_search_clicked(b):
    with output_area:
        clear_output()
        query = search_input.value.strip().lower()
        if query:
            results = [
                contact for contact in contacts
                if query in contact['name'].lower() or query in contact['phone'].lower()
            ]
            if results:
                print(f"--- Search Results for '{query}' ---")
                _display_contacts(results)
            else:
                print(f"No contacts found for '{query}'.")
        else:
            print("Please enter a search term.")
        search_input.value = ''

def on_clear_clicked(b):
    _clear_input_fields()
    with output_area:
        clear_output()
        print("Input fields cleared.")

def on_save_clicked(b):
    with output_area:
        clear_output()
        save_contacts_to_file()
        print("Contacts saved to 'contacts.txt'.")

def on_load_clicked(b):
    with output_area:
        clear_output()
        load_contacts_from_file()
        print("Contacts loaded from 'contacts.txt'.")

def on_export_csv_clicked(b):
    with output_area:
        clear_output()
        if not contacts:
            print("No contacts to export.")
            return

        csv_file_name = 'contacts.csv'
        try:
            with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
                fieldnames = ['name', 'phone', 'email']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                writer.writeheader()
                for contact in contacts:
                    writer.writerow(contact)
            print(f"Contacts exported to '{csv_file_name}' successfully.")
        except Exception as e:
            print(f"Error exporting contacts to CSV: {e}")

def on_contact_selected(change):
    with output_area:
        clear_output()
    selected_id = change.new
    if selected_id is not None:
        contact = contacts[selected_id]
        name_input.value = contact['name']
        phone_input.value = contact['phone']
        email_input.value = contact['email']
        contact_id_input.value = str(selected_id)
        with output_area:
            print(f"Contact ID {selected_id} selected: {contact['name']}")

#Conectar funciones a los botones y dropdown
add_button.on_click(on_add_clicked)
update_button.on_click(on_update_clicked)
delete_button.on_click(on_delete_clicked)
search_button.on_click(on_search_clicked)
clear_button.on_click(on_clear_clicked)
save_button.on_click(on_save_clicked)
load_button.on_click(on_load_clicked)
export_csv_button.on_click(on_export_csv_clicked)
contact_selector_dropdown.observe(on_contact_selected, names='value')

# Inicializar cargando contactos desde el archivo al inicio
load_contacts_from_file()

#Organizar los widgets en una interfaz única
input_widgets_layout = VBox([
    name_input,
    phone_input,
    email_input,
    contact_id_input,
    contact_selector_dropdown
])

button_row_1 = HBox([add_button, update_button, export_csv_button, delete_button])
button_row_2 = HBox([search_input, search_button, clear_button])
button_row_3 = HBox([save_button, load_button], layout=Layout(justify_content='center', width='auto'))

#Create a title widget
title_widget = HTML(value="<h1>Contact Management</h1>", layout=Layout(align_self='center'))

#Combinar todo en un solo VBox para la interfaz
full_interface = VBox([
    title_widget,
    input_widgets_layout,
    button_row_1,
    button_row_2,
    button_row_3,
    output_area
])

# Mostrar la interfaz completa
display(full_interface)